1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors



Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [7]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [8]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [9]:
df.describe()

,BGGId,Rating
count,1.894222e+07,1.894222e+07
mean,1.099533e+05,7.130957e+00
std,9.280217e+04,1.543023e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [10]:
any_null_values = df.isnull().any()
any_null_values

null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [11]:
df = df.dropna()

In [12]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [13]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [14]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [15]:
duplicates.describe()

count           1.0
mean     18942152.0
std             NaN
min      18942152.0
25%      18942152.0
50%      18942152.0
75%      18942152.0
max      18942152.0
dtype: float64

Remove duplicate ratings for the same game by the same user (due to different version of the game etc)

In [16]:
# Calculate the mean rating for each group of duplicate rows
mean_ratings = df.groupby(['Username', 'BGGId'])['Rating'].mean()

In [17]:
mean_ratings

Username        BGGId 
 Fu_Koios       112092    9.0
                223033    9.0
 beastvol       13        8.0
                118       7.0
                278       7.0
                         ... 
Æleksandr Þræð  2399      7.0
                2932      6.0
                5451      8.0
                5554      7.0
                9962      8.0
Name: Rating, Length: 18909465, dtype: float64

In [18]:
# Filter the DataFrame for the specific user and item
specific_user_item_ratings = df[(df['Username'] == 'hariseldoncwb') & (df['BGGId'] == 135382)]

# Calculate the mean rating for the specific user and item
mean_rating_specific_user_item = specific_user_item_ratings['Rating'].mean()
mean_rating_specific_user_item

5.5

In [19]:
# Merge the mean ratings back to the original DataFrame based on 'Username' and 'Name'
df_removed_duplicates = df.merge(mean_ratings, on=['Username', 'BGGId'], suffixes=('', '_mean'))

In [20]:
df_removed_duplicates

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [21]:
# Filter the DataFrame to keep only the original rows (non-duplicates) and the calculated mean ratings
df_unique = df_removed_duplicates[~df_removed_duplicates.duplicated(subset=['Username', 'BGGId'], keep=False)]

In [22]:
df_unique

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [23]:
# Drop the original 'Rating' column and rename the mean rating column
df_dropped = df_unique.drop(columns=['Rating']).rename(columns={'Rating_mean': 'Rating'})

In [24]:
# Reindex the columns to move 'Rating' to the second position
column_order = ['BGGId', 'Rating', 'Username', 'Name']
df_dropped = df_dropped.reindex(columns=column_order)

In [25]:
df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


In [26]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_dropped.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         15077
4           342
5         18152
          ...  
347146       42
347521       71
348955       67
349131       66
349161        7
Length: 21925, dtype: int64

Filter dataset

In [124]:
game_ratings = 2000

rating_counter = df_dropped['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df_dropped['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df_dropped.drop(index=df_dropped[games_mask].index, inplace=True)

df_dropped

,BGGId,Rating,Username,Name
849332,30549,4.0,robvw,Pandemic
849335,30549,4.0,bigcrab,Pandemic
849344,30549,4.0,ubuntu_tyler,Pandemic
849348,30549,4.0,Kbj032,Pandemic
849363,30549,4.0,dcarlton,Pandemic
...,...,...,...,...
15190186,180899,9.0,MeisterXC,Ponzi Scheme
15190189,180899,9.0,BlueWaldo,Ponzi Scheme
15190190,180899,9.0,rajeevbat,Ponzi Scheme
15190191,180899,9.0,guilou05,Ponzi Scheme


In [127]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_dropped.groupby('BGGId').size()
grouped_counts

BGGId
1          3790
3          9118
5          9486
10         5338
11        17273
          ...  
308765     2756
312484     7322
316554     6026
317985     3101
318977     2887
Length: 1093, dtype: int64

In [128]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_dropped.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-     193
-Johnny-     246
-mIDE-       355
-toni-       131
...Hammer    121
            ... 
zzyzxuk      117
zzzabiss     284
zzzhsm       169
zzzoren      136
zzzzzane     134
Length: 33722, dtype: int64

In [129]:
user_ratings = 100

rating_counter = df_dropped['Username'].value_counts()

filtered_out = rating_counter[rating_counter < user_ratings].index

user_mask = df_dropped['Username'].isin(filtered_out)

df_dropped.drop(index=df_dropped[user_mask].index, inplace=True)

df_dropped

,BGGId,Rating,Username,Name
849332,30549,4.0,robvw,Pandemic
849335,30549,4.0,bigcrab,Pandemic
849344,30549,4.0,ubuntu_tyler,Pandemic
849348,30549,4.0,Kbj032,Pandemic
849363,30549,4.0,dcarlton,Pandemic
...,...,...,...,...
15190186,180899,9.0,MeisterXC,Ponzi Scheme
15190189,180899,9.0,BlueWaldo,Ponzi Scheme
15190190,180899,9.0,rajeevbat,Ponzi Scheme
15190191,180899,9.0,guilou05,Ponzi Scheme


In [130]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_dropped.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-     193
-Johnny-     246
-mIDE-       355
-toni-       131
...Hammer    121
            ... 
zzyzxuk      117
zzzabiss     284
zzzhsm       169
zzzoren      136
zzzzzane     134
Length: 30372, dtype: int64

Train test split

In [131]:
# Define a function to split the data for each user
def user_split(data, test_size=0.2):    
    train_to_concat = []
    test_to_concat = []
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size, train_size=1-test_size)
        train_to_concat.append(train_user)
        test_to_concat.append(test_user)
    
    train = pd.concat(train_to_concat, ignore_index=True)
    test = pd.concat(test_to_concat, ignore_index=True)

    return train, test

# Split the data
train_set, test_set = user_split(df_dropped, test_size=0.2)


In [132]:
train_set

,BGGId,Rating,Username,Name
0,170216,8.0,-=Yod@=-,Blood Rage
1,143884,7.0,-=Yod@=-,Machi Koro
2,503,7.0,-=Yod@=-,Through the Desert
3,28143,7.5,-=Yod@=-,Race for the Galaxy
4,2381,7.5,-=Yod@=-,Scattergories
...,...,...,...,...
4599771,122522,5.0,zzzzzane,Smash Up
4599772,204583,8.0,zzzzzane,Kingdomino
4599773,176494,7.5,zzzzzane,Isle of Skye: From Chieftain to King
4599774,199561,7.8,zzzzzane,Sagrada


In [133]:
test_set

,BGGId,Rating,Username,Name
0,22141,7.5,-=Yod@=-,Cleopatra and the Society of Architects
1,46213,8.0,-=Yod@=-,Telestrations
2,102548,7.5,-=Yod@=-,Dungeon Fighter
3,20551,8.0,-=Yod@=-,Shogun
4,45315,8.0,-=Yod@=-,Dungeon Lords
...,...,...,...,...
1165229,36235,7.0,zzzzzane,The Duke
1165230,521,10.0,zzzzzane,Crokinole
1165231,155426,8.5,zzzzzane,Castles of Mad King Ludwig
1165232,161936,9.0,zzzzzane,Pandemic Legacy: Season 1


filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

In [134]:
# Pivot the DataFrame so a user matrix, so rows represent users and columns represent items
user_item_matrix = train_set.pivot(index='Username', columns='BGGId', values='Rating').fillna(0)



In [135]:
user_item_matrix

BGGId,1,3,5,10,11,12,13,15,18,28,...,296151,296912,297030,304420,306735,308765,312484,316554,317985,318977
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,0.0,0.0,0.0,0.0,7.0,0.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Johnny-,5.0,0.0,7.0,0.0,6.0,5.0,7.0,8.0,6.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-mIDE-,0.0,8.0,0.0,0.0,7.5,0.0,7.0,0.0,8.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,8.0
-toni-,0.0,9.0,0.0,0.0,7.0,0.0,4.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...Hammer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyzxuk,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0
zzzabiss,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
zzzhsm,5.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
# Transpose the user-item matrix to obtain an item-user matrix
item_user_matrix = user_item_matrix.T

In [137]:
item_user_matrix

Username,-=Yod@=-,-Johnny-,-mIDE-,-toni-,...Hammer,.JcK.,0010,0492372665,0b1_Ita,0ddjob,...,zzas,zzay,zzeroparticle,zztap,zzyzewitsch,zzyzxuk,zzzabiss,zzzhsm,zzzoren,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,5.0,0.0,0.0
3,0.0,0.0,8.0,9.0,0.0,6.0,0.0,0.0,7.0,0.0,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.3,0.0,0.0,6.0,5.0,0.0,0.0,8.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,7.0,6.0,7.5,7.0,0.0,0.0,0.0,0.0,7.5,9.0,...,0.0,6.6,6.0,8.0,0.0,0.0,0.0,0.0,7.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
312484,0.0,0.0,9.0,0.0,0.0,0.0,10.0,0.0,7.5,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,7.0,0.0,0.0,0.0
316554,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0


In [415]:
def calculate_cosine_similarity(matrix_transposed):
    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((matrix_transposed.shape[0], matrix_transposed.shape[0]))
    
    # Iterate over each row (item) in the transposed matrix by index
    for i in range(len(matrix_transposed)):
    #for i in range(1):
        for j in range(i, len(matrix_transposed)):  # Only compute upper triangle (similarity_matrix is symmetric)
            # Get rows by numeric position using iloc
            row_i = matrix_transposed.iloc[i]
            row_j = matrix_transposed.iloc[j]
            
            # Find indices where both rows have non-missing values
            non_missing_indices = np.logical_and(row_i != 0, row_j != 0)

            # Filter out ratings for common users
            common_ratings = non_missing_indices[non_missing_indices]
            
            if not common_ratings.empty:
                # Get non-missing values for common users
                common_usernames = common_ratings.index
                non_missing_values_i = row_i.loc[common_usernames].values.reshape(1, -1)
                non_missing_values_j = row_j.loc[common_usernames].values.reshape(1, -1)
                similarity = cosine_similarity(non_missing_values_i, non_missing_values_j)[0, 0]
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity
            
    return similarity_matrix

# Example usage:
# Assuming item_user_matrix_transposed is your transposed item-user matrix (DataFrame)
# similarity_matrix_items = custom_cosine_similarity_item(item_user_matrix_transposed)

In [142]:
from scipy.sparse import csr_matrix
import numpy as np

def calculate_cosine_similarity_4(matrix_transposed):

    sparse_matrix = csr_matrix(matrix_transposed.values)
    num_items = sparse_matrix.shape[0]

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((num_items, num_items))

    # Iterate over each row (item) in the transposed matrix by index
    #for i in range(10):
    for i in range(num_items):
        for j in range(i, num_items):  # Only compute upper triangle (similarity_matrix is symmetric)
            # Get rows by numeric position using csr_matrix row slicing
            row_i = sparse_matrix.getrow(i)
            row_j = sparse_matrix.getrow(j)

            # Find indices where both rows have non-missing values
            common_indices = np.intersect1d(row_i.indices, row_j.indices)

            if common_indices.size > 0:
                # Get non-missing values for common users
                non_missing_values_i = row_i[:, common_indices].toarray().flatten()
                non_missing_values_j = row_j[:, common_indices].toarray().flatten()
                similarity = cosine_similarity([non_missing_values_i], [non_missing_values_j])[0, 0]
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity

    return similarity_matrix



In [138]:
# Compute cosine similarity between items
item_similarity_10_of_1000 = calculate_cosine_similarity_4(item_user_matrix)

In [140]:
# Create a DataFrame with item names as both rows and columns
test_item_sim_df = pd.DataFrame(item_similarity_10_of_1000, index=item_user_matrix.index, columns=item_user_matrix.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(test_item_sim_df.values, 0)
test_item_sim_df

BGGId,1,3,5,10,11,12,13,15,18,28,...,296151,296912,297030,304420,306735,308765,312484,316554,317985,318977
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.976024,0.976694,0.972297,0.971950,0.974472,0.966181,0.950538,0.962231,0.957079,...,0.977480,0.978028,0.976444,0.977381,0.979579,0.981683,0.976668,0.979342,0.980633,0.980421
3,0.976024,0.000000,0.982356,0.976441,0.978930,0.983258,0.973638,0.957261,0.969645,0.963430,...,0.986014,0.985607,0.983781,0.985934,0.987329,0.987197,0.986765,0.987366,0.988434,0.985327
5,0.976694,0.982356,0.000000,0.976850,0.979940,0.979674,0.976087,0.961378,0.971003,0.962835,...,0.984654,0.982984,0.982093,0.984661,0.985046,0.986666,0.985351,0.985364,0.987013,0.984393
10,0.972297,0.976441,0.976850,0.000000,0.978832,0.973182,0.973756,0.958267,0.969182,0.956347,...,0.972357,0.974459,0.982015,0.980715,0.975551,0.977173,0.979687,0.977412,0.979668,0.980416
11,0.971950,0.978930,0.979940,0.978832,0.000000,0.977463,0.975332,0.958120,0.970382,0.963107,...,0.982817,0.983468,0.980119,0.982752,0.983901,0.983160,0.983214,0.982487,0.983906,0.982450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308765,0.981683,0.987197,0.986666,0.977173,0.983160,0.984429,0.977655,0.958477,0.975204,0.971922,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
312484,0.976668,0.986765,0.985351,0.979687,0.983214,0.986238,0.980213,0.963268,0.977294,0.966402,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
316554,0.979342,0.987366,0.985364,0.977412,0.982487,0.984919,0.979524,0.970281,0.977199,0.966685,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [143]:
# Compute cosine similarity between items
item_similarity = calculate_cosine_similarity_4(item_user_matrix)

In [144]:
# Create a DataFrame with item names as both rows and columns
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_similarity_df.values, 0)
item_similarity_df

BGGId,1,3,5,10,11,12,13,15,18,28,...,296151,296912,297030,304420,306735,308765,312484,316554,317985,318977
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.976024,0.976694,0.972297,0.971950,0.974472,0.966181,0.950538,0.962231,0.957079,...,0.977480,0.978028,0.976444,0.977381,0.979579,0.981683,0.976668,0.979342,0.980633,0.980421
3,0.976024,0.000000,0.982356,0.976441,0.978930,0.983258,0.973638,0.957261,0.969645,0.963430,...,0.986014,0.985607,0.983781,0.985934,0.987329,0.987197,0.986765,0.987366,0.988434,0.985327
5,0.976694,0.982356,0.000000,0.976850,0.979940,0.979674,0.976087,0.961378,0.971003,0.962835,...,0.984654,0.982984,0.982093,0.984661,0.985046,0.986666,0.985351,0.985364,0.987013,0.984393
10,0.972297,0.976441,0.976850,0.000000,0.978832,0.973182,0.973756,0.958267,0.969182,0.956347,...,0.972357,0.974459,0.982015,0.980715,0.975551,0.977173,0.979687,0.977412,0.979668,0.980416
11,0.971950,0.978930,0.979940,0.978832,0.000000,0.977463,0.975332,0.958120,0.970382,0.963107,...,0.982817,0.983468,0.980119,0.982752,0.983901,0.983160,0.983214,0.982487,0.983906,0.982450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308765,0.981683,0.987197,0.986666,0.977173,0.983160,0.984429,0.977655,0.958477,0.975204,0.971922,...,0.992450,0.988890,0.991536,0.992904,0.993538,0.000000,0.993043,0.993084,0.993065,0.991572
312484,0.976668,0.986765,0.985351,0.979687,0.983214,0.986238,0.980213,0.963268,0.977294,0.966402,...,0.992445,0.991068,0.991696,0.992654,0.993003,0.993043,0.000000,0.993583,0.993708,0.992438
316554,0.979342,0.987366,0.985364,0.977412,0.982487,0.984919,0.979524,0.970281,0.977199,0.966685,...,0.992083,0.990615,0.990032,0.992178,0.992807,0.993084,0.993583,0.000000,0.993554,0.991412


In [2]:
from scipy.sparse import csr_matrix
import numpy as np
from multiprocessing import Pool
from calculate_similarity import *

def calculate_cosine_similarity_4_parallel(matrix_transposed):
    sparse_matrix = csr_matrix(matrix_transposed.values)
    num_items = sparse_matrix.shape[0]
    similarity_matrix = np.zeros((num_items, num_items))

    with Pool() as pool:
        for i, result in enumerate(pool.imap_unordered(calculate_similarity_with_matrix, range(num_items), chunksize=10)):
            similarity_matrix[i] = result

    # Populate the lower triangle of the matrix (since it's symmetric)
    similarity_matrix = np.maximum(similarity_matrix, similarity_matrix.T)

    return similarity_matrix



In [349]:
from joblib import Parallel, delayed
from scipy.sparse import csr_matrix

def compute_similarity_pair(i, j, sparse_matrix, similarity_matrix):
    # Find indices where both rows have non-missing values
    non_missing_indices = np.logical_and(sparse_matrix[i].A != 0, sparse_matrix[j].A != 0)

    # Filter out ratings for common users
    common_ratings = non_missing_indices[non_missing_indices]

    # Compute cosine similarity for common ratings
    if np.any(common_ratings):
        # Extract common ratings
        common_ratings_i = sparse_matrix[i, non_missing_indices].toarray().reshape(1, -1)
        common_ratings_j = sparse_matrix[j, non_missing_indices].toarray().reshape(1, -1)
        similarity = cosine_similarity(common_ratings_i, common_ratings_j)[0, 0]
        similarity_matrix[i, j] = similarity
        similarity_matrix[j, i] = similarity

def calculate_cosine_similarity_2(matrix_transposed):
    # Convert DataFrame to sparse matrix representation
    sparse_matrix = csr_matrix(matrix_transposed.values)
    num_items = sparse_matrix.shape[0]


    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((num_items, num_items))

    # Parallelize the computation of cosine similarity
    Parallel(n_jobs=-1)(
        delayed(compute_similarity_pair)(i, j, sparse_matrix, similarity_matrix)
        for i in range(num_items) for j in range(i, num_items)
    )

    return similarity_matrix

# Example usage:
# similarity_matrix_items = calculate_cosine_similarity(item_user_matrix_transposed)

In [352]:
def compute_similarity_3(chunk):
    # Beräkna cosine similarity för den aktuella chunken
    return cosine_similarity(chunk.T)

def calc_sim_3(matrix):

    #Antal partitioner att dela upp datat i (kan justeras beroende på antal kärnor)
    num_partitions = 4

    #Dela upp datat i partitioner
    chunks = np.array_split(matrix.values, num_partitions)

    #Beräkna cosine similarity för varje partition parallellt
    similarities = Parallel(n_jobs=-1)(delayed(compute_similarity_3)(chunk) for chunk in chunks)

    #Sammanfoga resultaten från varje partition
    user_similarity_matrix = np.concatenate(similarities)

    #Skapa en DataFrame från similarity-matrisen
    user_similarity_df = pd.DataFrame(user_similarity_matrix, index=matrix.columns, columns=matrix.columns)

    return user_similarity_df


In [358]:
#Storlek på det nya datasetet
num_rows = 20
num_columns = 20

#Skapa det nya datasetet med slumpmässiga värden
data = pd.DataFrame(np.random.randint(0, 6, size=(num_rows, num_columns)),
                            index=[f'Item{i}' for i in range(1, num_rows + 1)],
                            columns=[f'Item{i}' for i in range(1, num_columns + 1)])

def compute_similarity(chunk):
    # Beräkna cosine similarity för den aktuella chunken
    return cosine_similarity(chunk.T)

#Antal partitioner att dela upp datat i (kan justeras beroende på antal kärnor)
num_partitions = 4

#Dela upp datat i partitioner
chunks = np.array_split(data.values, num_partitions)

#Beräkna cosine similarity för varje partition parallellt
similarities = Parallel(n_jobs=-1)(delayed(compute_similarity)(chunk) for chunk in chunks)

#Skapa en tom matris för att hålla den sammanfogade similarity-matrisen
merged_similarity_matrix = None

#Sammanfoga resultaten från varje partition
for similarity_matrix in similarities:
    if merged_similarity_matrix is None:
        merged_similarity_matrix = similarity_matrix
    else:
        merged_similarity_matrix += similarity_matrix

#Dela på antalet partitioner för att normalisera
merged_similarity_matrix /= num_partitions

#Skapa en DataFrame från similarity-matrisen
user_similarity_df = pd.DataFrame(merged_similarity_matrix, index=data.columns, columns=data.columns)

In [360]:
def item_cosine_similarity(data):
    # Beräkna cosine similarity mellan items
    similarity_matrix_items = cosine_similarity(data.T)
    # Skapa en DataFrame från similarity-matrisen
    item_similarity_df = pd.DataFrame(similarity_matrix_items, index=data.columns, columns=data.columns)
    return item_similarity_df

In [361]:
test_data_sim = item_cosine_similarity(data)

In [346]:
item_similarity_2 = calculate_cosine_similarity(item_user_matrix)

In [348]:
# Create a DataFrame with item names as both rows and columns
item_similarity_2_df = pd.DataFrame(item_similarity_2, index=item_user_matrix.index, columns=item_user_matrix.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_similarity_2_df.values, 0)
item_similarity_2_df

BGGId,1,2,3,4,5,6,7,8,9,10,...,344768,345036,345584,346482,346501,346703,346965,347521,348955,349131
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.970778,0.977473,0.975993,0.976472,0.94301,0.966028,0.969852,0.966274,0.972682,...,0.98473,0.991479,0.996858,0.980045,0.993698,0.981265,0.979682,0.889567,0.0,0.95389
2,0.970778,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
3,0.977473,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
4,0.975993,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
5,0.976472,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346703,0.981265,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
346965,0.979682,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
347521,0.889567,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000


In [402]:
test_calc_cos_4 = calculate_cosine_similarity_4(item_user_m_two_items)

In [404]:
# Create a DataFrame with item names as both rows and columns
test_calc_cos_4_df = pd.DataFrame(test_calc_cos_4, index=item_user_m_two_items.index, columns=item_user_m_two_items.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(test_calc_cos_4_df.values, 0)
test_calc_cos_4_df

BGGId,1,2
BGGId,,
1,0.000000,0.970778
2,0.970778,0.000000


In [408]:
test_calc_cos_4_1000_items = calculate_cosine_similarity_4(item_user_m_1000_items)

In [409]:
# Create a DataFrame with item names as both rows and columns
test_calc_cos_4_1000_items_df = pd.DataFrame(test_calc_cos_4_1000_items, index=item_user_m_1000_items.index, columns=item_user_m_1000_items.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(test_calc_cos_4_1000_items_df.values, 0)
test_calc_cos_4_1000_items_df

BGGId,1,2,3,4,5,6,7,8,9,10,...,1430,1431,1433,1434,1435,1436,1437,1439,1442,1443
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.970778,0.977473,0.975993,0.976472,0.943010,0.966028,0.969852,0.966274,0.972682,...,0.956193,0.957069,0.969531,0.957954,0.966487,0.927045,0.957181,0.954861,0.963507,0.968344
2,0.970778,0.000000,0.968632,0.979124,0.973407,0.936498,0.968991,0.965567,0.972447,0.972201,...,0.972867,0.985004,0.990703,0.957348,0.968205,0.919355,0.958494,0.947634,0.966776,0.966754
3,0.977473,0.968632,0.000000,0.976245,0.982731,0.958130,0.975239,0.968364,0.974367,0.977689,...,0.960979,0.970298,0.973359,0.966830,0.947133,0.950616,0.964533,0.955169,0.967744,0.974504
4,0.975993,0.979124,0.976245,0.000000,0.977390,0.937495,0.976188,0.975891,0.971335,0.976674,...,0.960612,0.980945,0.981252,0.916145,0.951672,0.930077,0.975481,0.988859,0.982763,0.948946
5,0.976472,0.973407,0.982731,0.977390,0.000000,0.955145,0.972783,0.967807,0.967392,0.975958,...,0.961293,0.965657,0.969011,0.960175,0.960714,0.945378,0.962569,0.961250,0.971405,0.967939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,0.927045,0.919355,0.950616,0.930077,0.945378,1.000000,0.955521,0.999703,0.975179,0.943094,...,0.965771,0.952422,1.000000,0.950425,0.951498,0.000000,0.961094,0.866764,0.955106,0.964755
1437,0.957181,0.958494,0.964533,0.975481,0.962569,1.000000,0.982365,0.000000,0.944262,0.963478,...,0.991601,0.956427,1.000000,0.916385,0.943443,0.961094,0.000000,0.988799,0.935100,0.000000
1439,0.954861,0.947634,0.955169,0.988859,0.961250,0.973012,0.963194,0.934395,0.963038,0.956922,...,0.966741,0.976039,0.988301,0.942540,0.950086,0.866764,0.988799,0.000000,0.978667,0.000000


In [ ]:
test_calc_cos_4_all = calculate_cosine_similarity_4(item_user_matrix)

In [4]:
test_calc_cos_4_1000_items_parallel = calculate_cosine_similarity_4_parallel(item_user_m_1000_items)

NameError: name 'item_user_m_1000_items' is not defined

In [326]:
def compute_similarity(i, j, matrix_transposed):
    # Get rows by numeric position using iloc
    row_i = matrix_transposed.iloc[i]
    #print("row i")
    #print(row_i)
    row_j = matrix_transposed.iloc[j]
    #print("row j")
    #print(row_j)
    non_missing_row_i = (row_i != 0)
    #print("non missing row i")
    #print(non_missing_row_i)
    
    non_missing_row_j = (row_j != 0)
    #print("non missing row j")
    #print(non_missing_row_j)

    # Find indices where both rows have non-missing values
    non_missing_indices = non_missing_row_i & non_missing_row_j

    # Filter out ratings for common users
    common_ratings = non_missing_indices[non_missing_indices]
    #print("common ratings")
    #print(common_ratings)

    if not common_ratings.empty:

        # Get non-missing values for common users
        common_usernames = common_ratings.index
        non_missing_values_i = row_i.loc[common_usernames].values.reshape(1, -1)
        non_missing_values_j = row_j.loc[common_usernames].values.reshape(1, -1)
        similarity = cosine_similarity(non_missing_values_i, non_missing_values_j)[0, 0]
        return (i, j, similarity)
    else:
        return (i, j, 0.0)

In [383]:
users = ['User' + str(i) for i in range(1, 7)]
films = ['Film' + str(i) for i in range(1, 6)]
df_movie = pd.DataFrame(index=users, columns=films)
# #Fyll DataFrame med slumpmässiga betyg från 0 till 5
# for user in users:
#     for film in films:
#         # Slumpmässigt välj ett betyg från 0 till 5, där 0 indikerar att användaren inte har betygsatt filmen
#         rating = np.random.choice([0, 1, 2, 3, 4, 5])
#         df_movie.at[user, film] = int(rating)
# Generate random ratings matrix
ratings_matrix = np.random.randint(0, 6, size=(len(users), len(films)))

# Create DataFrame
df_movie = pd.DataFrame(ratings_matrix, index=users, columns=films)
df_movie = df_movie.T
df_movie

,User1,User2,User3,User4,User5,User6
Film1,1,3,2,0,5,3
Film2,3,4,0,5,2,5
Film3,5,0,5,3,5,0
Film4,3,3,0,0,3,1
Film5,1,5,5,0,1,4


In [336]:
test_sim = custom_cosine_similarity_item(df_movie)
test_sim

array([[1.        , 0.92031569, 0.7581754 , 0.8674724 , 0.92031569],
       [0.92031569, 1.        , 0.73029674, 0.75887138, 0.8       ],
       [0.7581754 , 0.73029674, 1.        , 0.95405567, 0.77459667],
       [0.8674724 , 0.75887138, 0.95405567, 1.        , 0.91473965],
       [0.92031569, 0.8       , 0.77459667, 0.91473965, 1.        ]])

In [339]:
# Create a DataFrame with item names as both rows and columns
test_sim_df = pd.DataFrame(test_sim, index=df_movie.index, columns=df_movie.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(test_sim_df.values, 0)
test_sim_df

,Film1,Film2,Film3,Film4,Film5
Film1,0.000000,0.920316,0.758175,0.867472,0.920316
Film2,0.920316,0.000000,0.730297,0.758871,0.800000
Film3,0.758175,0.730297,0.000000,0.954056,0.774597
Film4,0.867472,0.758871,0.954056,0.000000,0.914740
Film5,0.920316,0.800000,0.774597,0.914740,0.000000


In [327]:
test_similarity = compute_similarity(0, 1, item_user_matrix)
test_similarity

common ratings
Username
AATLEMIDRM    True
Aldie         True
Axiin         True
Bankler       True
Be4tngU       True
              ... 
sloehand      True
smilingra     True
tinderfire    True
tonycrider    True
yolandavi     True
Length: 62, dtype: bool


(0, 1, 0.9707779918368868)

In [188]:
item_similarity

array([[1.        , 0.04224476, 0.23615647, ..., 0.00357582, 0.        ,
        0.02111222],
       [0.04224476, 1.        , 0.05627787, ..., 0.00600601, 0.        ,
        0.        ],
       [0.23615647, 0.05627787, 1.        , ..., 0.01393259, 0.00442959,
        0.01838933],
       ...,
       [0.00357582, 0.00600601, 0.01393259, ..., 1.        , 0.00244807,
        0.        ],
       [0.        , 0.        , 0.00442959, ..., 0.00244807, 1.        ,
        0.        ],
       [0.02111222, 0.        , 0.01838933, ..., 0.        , 0.        ,
        1.        ]])

In [189]:
# Create a DataFrame with item names as both rows and columns
item_item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels

In [206]:
item_item_similarity_df

BGGId,1,2,3,4,5,6,7,8,9,10,...,344768,345036,345584,346482,346501,346703,346965,347521,348955,349131
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.042245,0.236156,0.110157,0.245028,0.051265,0.095172,0.075352,0.166468,0.186407,...,0.014651,0.012133,0.008354,0.017105,0.016574,0.020744,0.013029,0.003576,0.000000,0.021112
2,0.042245,0.000000,0.056278,0.052978,0.073469,0.033542,0.064748,0.034108,0.051368,0.065613,...,0.000000,0.000000,0.000000,0.000000,0.006394,0.004259,0.000000,0.006006,0.000000,0.000000
3,0.236156,0.056278,0.000000,0.083479,0.279624,0.035158,0.124568,0.043962,0.157062,0.239486,...,0.021121,0.014278,0.010166,0.012980,0.015752,0.031575,0.028952,0.013933,0.004430,0.018389
4,0.110157,0.052978,0.083479,0.000000,0.083748,0.143612,0.089438,0.129525,0.146013,0.114824,...,0.003602,0.011108,0.000000,0.020700,0.030562,0.014934,0.014835,0.000000,0.000000,0.030070
5,0.245028,0.073469,0.279624,0.083748,0.000000,0.037900,0.139019,0.057608,0.135014,0.221985,...,0.014726,0.014614,0.009744,0.014583,0.008907,0.025560,0.022768,0.013641,0.002519,0.010941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346703,0.020744,0.004259,0.031575,0.014934,0.025560,0.000000,0.017211,0.005496,0.018335,0.034212,...,0.009214,0.026771,0.018521,0.044227,0.035461,0.000000,0.044462,0.000000,0.000000,0.047510
346965,0.013029,0.000000,0.028952,0.014835,0.022768,0.000000,0.011379,0.009311,0.023415,0.033920,...,0.079830,0.037451,0.034106,0.063336,0.081484,0.044462,0.000000,0.000000,0.000000,0.146388
347521,0.003576,0.006006,0.013933,0.000000,0.013641,0.000000,0.005891,0.000000,0.006905,0.006911,...,0.019054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002448,0.000000


In [ ]:
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_item_similarity_df.values, 0)

In [395]:
test_item_sim_4 = calculate_cosine_similarity_4(df_movie)
test_item_sim_4

array([[1.        , 0.82060994, 0.84327404, 0.85470432, 0.75265055],
       [0.82060994, 1.        , 0.84477721, 0.8229512 , 0.93385921],
       [0.84327404, 0.84477721, 1.        , 1.        , 0.77777778],
       [0.85470432, 0.8229512 , 1.        , 1.        , 0.72048802],
       [0.75265055, 0.93385921, 0.77777778, 0.72048802, 1.        ]])

In [397]:
# Create a DataFrame with item names as both rows and columns
test_item_sim_4_df = pd.DataFrame(test_item_sim_4, index=df_movie.index, columns=df_movie.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(test_item_sim_4_df.values, 0)
test_item_sim_4_df

,Film1,Film2,Film3,Film4,Film5
Film1,0.000000,0.820610,0.843274,0.854704,0.752651
Film2,0.820610,0.000000,0.844777,0.822951,0.933859
Film3,0.843274,0.844777,0.000000,1.000000,0.777778
Film4,0.854704,0.822951,1.000000,0.000000,0.720488
Film5,0.752651,0.933859,0.777778,0.720488,0.000000


In [215]:
# Convert similarity to distances (inversely proportional to similarities)
distance_matrix = 1 - item_item_similarity_df

In [216]:
distance_matrix

BGGId,1,2,3,4,5,6,7,8,9,10,...,344768,345036,345584,346482,346501,346703,346965,347521,348955,349131
BGGId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.957755,0.763844,0.889843,0.754972,0.948735,0.904828,0.924648,0.833532,0.813593,...,0.985349,0.987867,0.991646,0.982895,0.983426,0.979256,0.986971,0.996424,1.000000,0.978888
2,0.957755,1.000000,0.943722,0.947022,0.926531,0.966458,0.935252,0.965892,0.948632,0.934387,...,1.000000,1.000000,1.000000,1.000000,0.993606,0.995741,1.000000,0.993994,1.000000,1.000000
3,0.763844,0.943722,1.000000,0.916521,0.720376,0.964842,0.875432,0.956038,0.842938,0.760514,...,0.978879,0.985722,0.989834,0.987020,0.984248,0.968425,0.971048,0.986067,0.995570,0.981611
4,0.889843,0.947022,0.916521,1.000000,0.916252,0.856388,0.910562,0.870475,0.853987,0.885176,...,0.996398,0.988892,1.000000,0.979300,0.969438,0.985066,0.985165,1.000000,1.000000,0.969930
5,0.754972,0.926531,0.720376,0.916252,1.000000,0.962100,0.860981,0.942392,0.864986,0.778015,...,0.985274,0.985386,0.990256,0.985417,0.991093,0.974440,0.977232,0.986359,0.997481,0.989059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346703,0.979256,0.995741,0.968425,0.985066,0.974440,1.000000,0.982789,0.994504,0.981665,0.965788,...,0.990786,0.973229,0.981479,0.955773,0.964539,1.000000,0.955538,1.000000,1.000000,0.952490
346965,0.986971,1.000000,0.971048,0.985165,0.977232,1.000000,0.988621,0.990689,0.976585,0.966080,...,0.920170,0.962549,0.965894,0.936664,0.918516,0.955538,1.000000,1.000000,1.000000,0.853612
347521,0.996424,0.993994,0.986067,1.000000,0.986359,1.000000,0.994109,1.000000,0.993095,0.993089,...,0.980946,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.997552,1.000000


Calculate nearest neighbor

In [218]:
k = 20

# Create a NearestNeighbors object
nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')

In [219]:
# Fit the model using the item-item similarity matrix
nearest_neighbors_model.fit(distance_matrix.values)

NearestNeighbors(metric='precomputed', n_neighbors=20)

In [245]:
BGGId_80006 = 80006
BGGId_2 = 2

# Find the index of the item
item_index = distance_matrix.index.get_loc(BGGId_2)

In [246]:
item_index

1

In [247]:
# Find the indices of the nearest neighbors
distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix.iloc[item_index]], return_distance=True)

In [248]:
distances

array([[0.87657606, 0.90033503, 0.90437225, 0.90704539, 0.91288778,
        0.92010317, 0.92037898, 0.92104781, 0.92106783, 0.92178282,
        0.92219941, 0.92239426, 0.92273396, 0.92311167, 0.92382116,
        0.92387254, 0.9241849 , 0.92426584, 0.92506973, 0.92522267]])

In [249]:
neighbor_indices

array([[  28,  652,  296,  802, 1332,  627,   84,   97,  170, 6697,  902,
         263, 2613,  717, 3283,  411,   90,   92,  525, 1803]])

In [250]:
#BGGId_item_1 = 2
#BGGId_item_2 = 9830

# Retrieve the indices of the items in the DataFrame
#item1_index = item_item_similarity_df.index.get_loc(BGGId_item_1)  # Replace 'item2' with the name of the first item
#item2_index = item_item_similarity_df.index.get_loc(BGGId_item_2)   # Replace '3981' with the name of the second item

# Retrieve the distance between the two items from the distance matrix
distance_between_items = distance_matrix.iloc[1, 28]
distance_between_items

0.8765760649125957

In [251]:
item_item_similarity_between_items = item_item_similarity_df.iloc[1, 28]
item_item_similarity_between_items

0.1234239350874043

In [252]:
def find_nearest_neighbors(distance_matrix, k, BGGId):

    # Create a NearestNeighbors object
    nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')

    # Fit the model using the item-item similarity matrix
    nearest_neighbors_model.fit(distance_matrix.values)

    # Find the index of the item
    item_index = distance_matrix.index.get_loc(BGGId)

    # Find the distance and indices of the nearest neighbors
    distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix.iloc[item_index]], return_distance=True)

    return distances, neighbor_indices

Här nedan skriver vi funktionen som tar input data, user, k, top_n

5 Function RecommendItems(data, user, k, top N):
6 predicted ratings ← [];
7 for each item not in user.items do
8 N u ← set of k items similar to item that user u has rated;
9 numerator ← 0;
10 denominator ← 0;
11 for each j in N u do
12 numerator ← numerator + data[item][j] × user.items[j].rating;
13 denominator ← denominator + |data[item][j]|;
14 predicted ratings[item] ← numerator denominator ;
15 Sort predicted ratings by predicted rating in descending order;
16 recommended items ← predicted ratings;
17 recommended items ← recommended items[: top N ];
18 return recommended items

In [267]:
def get_user_items(username):
    # Get the column index for the specified user
    user_index = item_user_matrix.columns.get_loc(username)
    
    # Filter out the rows where the user has interactions (non-null values)
    user_items = item_user_matrix.iloc[:, user_index].dropna().index.tolist()


    return user_items



In [268]:
user_items = get_user_items("-=Yod@=-")
user_items

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 66,
 67,
 68,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 82,
 83,
 84,
 85,
 87,
 88,
 89,
 90,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 102,
 103,
 104,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 124,
 125,
 126,
 127,
 128,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 148,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 188,
 189,
 190,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 204,
 206,
 207,
 208,
 210,
 211,
 214,
 215,

In [ ]:
def recommend_items(distance_matrix, username, k, top_N):

    predicted_ratings = []

    

    


Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 